# Dask Bags


Dask Bag implements operations like `map`, `filter`, `groupby` and aggregations on collections of Python objects. It does this in parallel and in small memory using Python iterators. It is similar to a parallel version of itertools or a Pythonic version of the PySpark RDD.

Dask Bags are often used to do simple preprocessing on log files, JSON records, or other user defined Python objects.

Full API documentation is available here: http://docs.dask.org/en/latest/bag-api.html

## Create Random Data

We create a random set of record data and store it to disk as many JSON files.  This will serve as our data for this notebook.

In [1]:
import dask
import json
import os

os.makedirs('data', exist_ok=True)              # Create data/ directory

b = dask.datasets.make_people()                 # Make records of people
b.map(json.dumps).to_textfiles('data/*.json')   # Encode as JSON, write to disk

['/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/0.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/1.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/2.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/3.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/4.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/5.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/6.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/7.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/8.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/9.json']

## Read JSON data

Now that we have some JSON data in a file lets take a look at it with Dask Bag and Python JSON module. Dask Bag can load data directly from text files using `db.read_text` (use `db.from_sequence` to create a bag from an existing Python iterable)

In [2]:
!head -n 2 data/0.json

{"age": 43, "name": ["Refugio", "Bailey"], "occupation": "Warehouseman", "telephone": "(782) 313-8330", "address": {"address": "877 Friendship Spur", "city": "Appleton"}, "credit-card": {"number": "2694 7084 9153 2441", "expiration-date": "03/24"}}
{"age": 31, "name": ["Daryl", "Strickland"], "occupation": "Bricklayer", "telephone": "(886) 704-5628", "address": {"address": "1128 Jessie West Circle", "city": "SeaTac"}, "credit-card": {"number": "5550 5705 3841 9164", "expiration-date": "06/23"}}


In [3]:
import dask.bag as db
import json

b = db.read_text('data/*.json').map(json.loads)
b

dask.bag<loads, npartitions=10>

In [4]:
b.take(2)

({'age': 43,
  'name': ['Refugio', 'Bailey'],
  'occupation': 'Warehouseman',
  'telephone': '(782) 313-8330',
  'address': {'address': '877 Friendship Spur', 'city': 'Appleton'},
  'credit-card': {'number': '2694 7084 9153 2441',
   'expiration-date': '03/24'}},
 {'age': 31,
  'name': ['Daryl', 'Strickland'],
  'occupation': 'Bricklayer',
  'telephone': '(886) 704-5628',
  'address': {'address': '1128 Jessie West Circle', 'city': 'SeaTac'},
  'credit-card': {'number': '5550 5705 3841 9164',
   'expiration-date': '06/23'}})

## Map, Filter, Aggregate

We can process this data by filtering out only certain records of interest, mapping functions over it to process our data, and aggregating those results to a total value.

In [5]:
b.filter(lambda record: record['age'] > 30).take(2)  # Select only people over 30

({'age': 43,
  'name': ['Refugio', 'Bailey'],
  'occupation': 'Warehouseman',
  'telephone': '(782) 313-8330',
  'address': {'address': '877 Friendship Spur', 'city': 'Appleton'},
  'credit-card': {'number': '2694 7084 9153 2441',
   'expiration-date': '03/24'}},
 {'age': 31,
  'name': ['Daryl', 'Strickland'],
  'occupation': 'Bricklayer',
  'telephone': '(886) 704-5628',
  'address': {'address': '1128 Jessie West Circle', 'city': 'SeaTac'},
  'credit-card': {'number': '5550 5705 3841 9164',
   'expiration-date': '06/23'}})

In [6]:
b.map(lambda record: record['occupation']).take(2)  # Select the occupation field

('Warehouseman', 'Bricklayer')

In [7]:
b.count().compute()  # Count total number of records

10000

## Chain computations

It is common to do many of these steps in one pipeline, only calling `compute` or `take` at the end.

In [8]:
result = (b.filter(lambda record: record['age'] > 30)
           .map(lambda record: record['occupation'])
           .frequencies(sort=True)
           .topk(10, key=1))
result

dask.bag<topk-aggregate, npartitions=1>

As with all lazy Dask collections, we need to call `compute` to actually evaluate our result.  The `take` method used in earlier examples is also like `compute` and will also trigger computation.

In [9]:
result.compute()

[('Professional Wrestler', 18),
 ('Body Fitter', 17),
 ('Train Driver', 15),
 ('Economist', 14),
 ('Wine Merchant', 14),
 ('Building Foreman', 13),
 ('Chaplain', 13),
 ('Licensed Premises', 13),
 ('Laminator', 13),
 ('Project Engineer', 13)]

## Transform and Store

Sometimes we want to compute aggregations as above, but sometimes we want to store results to disk for future analyses.  For that we can use methods like `to_textfiles` and `json.dumps`, or we can convert to Dask Dataframes and use their storage systems, which we'll see more of in the next section.

In [12]:
(b.filter(lambda record: record['age'] > 30)  # Select records of interest
  .map(json.dumps)                            # Convert Python objects to text
  .to_textfiles('data/processed.*.json'))     # Write to local disk

['/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/processed.0.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/processed.1.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/processed.2.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/processed.3.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/processed.4.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/processed.5.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/processed.6.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/processed.7.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/processed.8.json',
 '/home/jovyan/work/SOSC23-livesessions/day2/Dask/data/processed.9.json']

## Convert to Dask Dataframes

Dask Bags are good for reading in initial data, doing a bit of pre-processing, and then handing off to some other more efficient form like Dask Dataframes.  Dask Dataframes use Pandas internally, and so can be much faster on numeric data and also have more complex algorithms.  

However, Dask Dataframes also expect data that is organized as flat columns.  It does not support nested JSON data very well (Bag is better for this).

Here we make a function to flatten down our nested data structure, map that across our records, and then convert that to a Dask Dataframe.

In [13]:
b.take(1)

({'age': 43,
  'name': ['Refugio', 'Bailey'],
  'occupation': 'Warehouseman',
  'telephone': '(782) 313-8330',
  'address': {'address': '877 Friendship Spur', 'city': 'Appleton'},
  'credit-card': {'number': '2694 7084 9153 2441',
   'expiration-date': '03/24'}},)

In [14]:
def flatten(record):
    return {
        'age': record['age'],
        'occupation': record['occupation'],
        'telephone': record['telephone'],
        'credit-card-number': record['credit-card']['number'],
        'credit-card-expiration': record['credit-card']['expiration-date'],
        'name': ' '.join(record['name']),
        'street-address': record['address']['address'],
        'city': record['address']['city']   
    }

b.map(flatten).take(1)

({'age': 43,
  'occupation': 'Warehouseman',
  'telephone': '(782) 313-8330',
  'credit-card-number': '2694 7084 9153 2441',
  'credit-card-expiration': '03/24',
  'name': 'Refugio Bailey',
  'street-address': '877 Friendship Spur',
  'city': 'Appleton'},)

In [15]:
df = b.map(flatten).to_dataframe()
df.head()

,age,occupation,telephone,credit-card-number,credit-card-expiration,name,street-address,city
0,43,Warehouseman,(782) 313-8330,2694 7084 9153 2441,03/24,Refugio Bailey,877 Friendship Spur,Appleton
1,31,Bricklayer,(886) 704-5628,5550 5705 3841 9164,06/23,Daryl Strickland,1128 Jessie West Circle,SeaTac
2,32,Milklady,163-232-0941,4664 1472 8163 9425,09/16,Spencer Bates,750 Grote Crossing,Harlingen
3,65,Economist,881.547.1361,3704 249531 54006,03/22,Johanne Fields,580 Diamond Trace,La Vergne
4,49,Word Processing Operator,503-941-3916,4187 7341 2369 1991,03/23,Lisabeth Spence,765 Lyon Spur,Agoura Hills


We can now perform the same computation as before, but now using Pandas and Dask dataframe.

In [16]:
df[df.age > 30].occupation.value_counts().nlargest(10).compute()

occupation
Professional Wrestler    18
Body Fitter              17
Train Driver             15
Economist                14
Wine Merchant            14
Chaplain                 13
Project Engineer         13
Licensed Premises        13
Building Foreman         13
Laminator                13
Name: count, dtype: int64

## Learn More

You may be interested in the following links:

-  [Dask Bag Documentation](https://docs.dask.org/en/latest/bag.html)
-  [API Documentation](http://docs.dask.org/en/latest/bag-api.html)